In [1]:
import random
import torch

from utils.Logger import showPlot
from utils.Timer import asMinutes, timeSince

%load_ext autoreload
from data.exampleJP_Data import Lang, prepareData
from model.rnn_model.encoderRNN import EncoderGRU, EncoderLSTM
from model.rnn_model.decoderRNN import (
    DecoderGRU, DecoderLSTM, 
    AttnDecoderGRU, AttnDecoderLSTM1, AttnDecoderLSTM2
)
from model.seq2seq_Model import (
    Seq2Seq_batch_ptModel, 
    Seq2SeqTranslate_ptTokenizer,
)
from baselineJP_ExpTrain import example_ExpTrain
%autoreload

In [2]:
DATA_DIR = '_data_example'

## load data

In [3]:
if True:    # 解凍する場合は True に書き換え
    !mkdir  $DATA_DIR
    %cd  $DATA_DIR

    !git clone https://github.com/odashi/small_parallel_enja
    !echo '------------------'
    !head -n 5 small_parallel_enja/train.en
    !head -n 5 small_parallel_enja/train.ja
    !echo '------------------'

    !ls
    %cd ../

mkdir: _data_example: File exists
/Volumes/GoogleDrive/マイドライブ/__datasets__/[CV][NLP]「センター試験xml」/annotate_img/datas/center_exam_2011/construct/Experiment/_data_example
fatal: destination path 'small_parallel_enja' already exists and is not an empty directory.
------------------
i can 't tell who will arrive first .
many animals have been destroyed by men .
i 'm in the tennis club .
emi looks happy .
please bear this fact in mind .
誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
多く の 動物 が 人間 に よ っ て 滅ぼ さ れ た 。
私 は テニス 部員 で す 。
エミ は 幸せ そう に 見え ま す 。
この 事実 を 心 に 留め て お い て 下さ い 。
------------------
data                data.zip            small_parallel_enja
/Volumes/GoogleDrive/マイドライブ/__datasets__/[CV][NLP]「センター試験xml」/annotate_img/datas/center_exam_2011/construct/Experiment


## setup Experiment

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## setup Data

In [5]:
input_lang, output_lang, pairs = prepareData(
    'en', 'ja', f'{DATA_DIR}/small_parallel_enja', False)

Reading lines...
Read 50000 sentence pairs
Counting words...
Counted words:
en 6637
ja 8777


In [6]:
print(random.choice(pairs))
print(input_lang.n_words, output_lang.n_words)

['this bus will take you to the station .', 'この バス に 乗 れ ば 駅 に 行け ま す 。']
6637 8777


In [7]:
# train / test split
from sklearn.model_selection import train_test_split
train_pairs, test_pairs = train_test_split(pairs, test_size=0.2)

## setup Model

In [8]:
## Model
tokenizer = Seq2SeqTranslate_ptTokenizer(
                    input_lang, output_lang, device)

### test

In [9]:
batch_size = 10
emb_size = 8
hid_size = 12
MAX_LENGTH = 18

test_encoder = EncoderLSTM(input_lang.n_words, emb_size, hid_size)
test_decoder1 = AttnDecoderLSTM1(
                    emb_size, hid_size, output_lang.n_words,
                    device, max_length=MAX_LENGTH)
seq2seq_test_model = Seq2Seq_batch_ptModel(
                    tokenizer, device,
                    dropout_p=0.1, max_length=MAX_LENGTH)
seq2seq_test_model.set_models(test_encoder, test_decoder1)
seq2seq_test_model.exec_test(train_pairs, batch_size=batch_size)


[Info] input_batch.shape, input_lens.shape
     =  torch.Size([10, 10]) torch.Size([10])
[Info] enc_outputs.shape, hidden_h.shape, hidden_c.shape
     =  torch.Size([10, 10, 24]) torch.Size([1, 10, 12]) torch.Size([1, 10, 12])
[Info] hidden[0].shape, hidden[1].shape
     =  torch.Size([10, 12]) torch.Size([10, 12])
[Info] dec_outputs.shape, hidden[0].shape, hidden[1].shape, attn_weights.shape
     =  torch.Size([10, 8777]) torch.Size([10, 12]) torch.Size([10, 12]) torch.Size([10, 18])
[Info] loss.item()
     =  9.123669624328613


In [10]:
attn_size = 9

test_decoder2 = AttnDecoderLSTM2(
                    emb_size, hid_size, attn_size, 
                    output_lang.n_words, device).to(device)
seq2seq_test_model.set_models(test_encoder, test_decoder2)
seq2seq_test_model.exec_test(train_pairs, batch_size=batch_size)

[Info] input_batch.shape, input_lens.shape
     =  torch.Size([12, 10]) torch.Size([10])
[Info] enc_outputs.shape, hidden_h.shape, hidden_c.shape
     =  torch.Size([12, 10, 24]) torch.Size([1, 10, 12]) torch.Size([1, 10, 12])
[Info] hidden[0].shape, hidden[1].shape
     =  torch.Size([10, 12]) torch.Size([10, 12])
[Info] dec_outputs.shape, hidden[0].shape, hidden[1].shape, attn_weights.shape
     =  torch.Size([10, 8777]) torch.Size([10, 12]) torch.Size([10, 12]) torch.Size([10, 12])
[Info] loss.item()
     =  9.075693130493164


## setup Model

In [12]:
%load_ext autoreload
from model.rnn_model.decoderRNN import (
    DecoderGRU, DecoderLSTM, 
    AttnDecoderGRU, AttnDecoderLSTM1, AttnDecoderLSTM2
)
from model.seq2seq_Model import (
    Seq2Seq_batch_ptModel, 
    Seq2SeqTranslate_ptTokenizer,
)
from baselineJP_ExpTrain import example_ExpTrain
%autoreload


emb_size = 1024
hidden_size = 1024

encoder = EncoderLSTM(input_lang.n_words, emb_size, hidden_size)
encoder.load_weights(
            load_m_dir='_logs/2021-05-05_21h02m20s', 
            load_m_file_name='encoder.pth')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[info] encoder.pth loaded !


## setup exec

In [12]:
%load_ext autoreload
from baselineJP_ExpTrain import example_ExpTrain
%autoreload

decoder1 = AttnDecoderLSTM1(emb_size, hidden_size, output_lang.n_words, device)
decoder1.load_weights(
            load_m_dir='_logs/2021-05-05_21h02m20s', 
            load_m_file_name='decoder.pth')
seq2seq_lstm1_model = Seq2Seq_batch_ptModel(
                            tokenizer, device,
                            dropout_p=0.1, max_length=MAX_LENGTH,
                            save_m_dir='_logs', 
                            save_m_file_names=('encoder.pth', 'decoder.pth'))
seq2seq_lstm1_model.set_models(encoder, decoder1)

## exec
exp_train = example_ExpTrain(train_pairs, test_pairs)
exp_train.exec(seq2seq_lstm1_model, 
                epochs=20, batch_size=200, 
                teacher_forcing=0.9, early_stopping=3)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0it [00:13, ?it/s]
0it [00:00, ?it/s]/Users/m_ryo/.pyenv/versions/3.8.4/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/m_ryo/.pyenv/versions/3.8.4/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/m_ryo/.pyenv/versions/3.8.4/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overl

KeyboardInterrupt: 

In [36]:
evaluate_batch_randomly(seq2seq_model, test_pairs, n=5)

> how do you like my new suit ?
= 私 の 新し い 服 は いかが ？
< 彼 は は は は は は 。 <EOS>
bleu: 1.0832677820940877e-231

/Users/m_ryo/.pyenv/versions/3.8.4/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/m_ryo/.pyenv/versions/3.8.4/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/m_ryo/.pyenv/versions/3.8.4/lib/python3.8/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore th

RuntimeError: In set_text: Could not load glyph (error code 0x55)

In [ ]:
decoder2 = AttnDecoderLSTM2(emb_size, hidden_size, output_lang.n_words, device)
seq2seq_lstm2_model = Seq2Seq_batch_ptModel(
                            tokenizer, device,
                            dropout_p=0.1, max_length=MAX_LENGTH)
seq2seq_lstm2_model.set_models(encoder, decoder2)

## exec
exp_train = example_ExpTrain(train_pairs, test_pairs)
exp_train.exec(seq2seq_lstm2_model, 
                epochs=20, batch_size=200, 
                teacher_forcing=0.9, early_stopping=3)

In [ ]:
evaluate_batch_randomly(seq2seq_lstm2_model, test_pairs, n=10)